In [1]:
# import relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
# load the data

ec_df = pd.read_csv(r"/Users/adityamxr/Desktop/Electricity Demand Projection Project/data/Electricity Consumption.csv")

In [3]:
# verify import 

ec_df.head()

,DATE,Electricty_Consumption_in_TW
0,1/1/1973,35.9728
1,2/1/1973,36.1334
2,3/1/1973,35.0625
3,4/1/1973,33.8416
4,5/1/1973,33.5107


### Basic EDA

In [4]:
# check dataframe info
ec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   DATE                          561 non-null    object 
 1   Electricty_Consumption_in_TW  561 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.9+ KB


**No nulls present**

In [5]:
ec_df['DATE'] = ec_df['DATE'].astype(str)  # ensure it's treated as a string before cleaning
ec_df['DATE'] = ec_df['DATE'].str.strip()  # remove any leading or trailing spaces

In [8]:
# convert date column to datetime64ns

# try converting the DATE column again without specifying the format
ec_df['DATE'] = pd.to_datetime(ec_df['DATE'], errors='coerce')

# check if the conversion worked and if there are any NaT values left
print('Number of NaT values: ', ec_df['DATE'].isnull().sum()) 

# drop any rows where DATE conversion failed (if NaT values exist)
ec_df = ec_df.dropna(subset=['DATE'])

# set the DATE column as the index for time series analysis
ec_df.set_index('DATE', inplace=True)

# check the first few rows to confirm the changes
print(ec_df.head())

Number of NaT values:  0
            Electricty_Consumption_in_TW
DATE                                    
1973-01-01                       35.9728
1973-02-01                       36.1334
1973-03-01                       35.0625
1973-04-01                       33.8416
1973-05-01                       33.5107


#### Plot the data to visualize the time series components.